In [10]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [11]:
dataset = 'emma' # onion or emma
input_data_dir = f'../data/'

features = {
    'emotion_embeddings': ['id_emma.tsv', 'id_gems.tsv'],
    'audio_embeddings': ['id_maest.tsv'],
    'textual_embeddings': ['id_bert.tsv'],
    'visual_embeddings': ['id_resnet.tsv']
}

output_dir = f'../dataset/{dataset}/'

os.makedirs(output_dir, exist_ok=True)

In [12]:
df_interactions = pd.read_csv(f'../data/userid_trackid_timestamp_{dataset}.tsv', sep='\t')
    
df_interactions['rating'] = 5
sample_items = df_interactions['track_id'].unique()

# map item and users to numbers
df_interactions['user_id_int'] = df_interactions['user_id'].astype('category').cat.codes
df_interactions['track_id_int'] = df_interactions['track_id'].astype('category').cat.codes


print(df_interactions.shape)
df_interactions.head()

(7916, 7)


,user_id,track_id,timestamp,period,rating,user_id_int,track_id_int
0,34658,Q08DjqYYbc3AvWQx,2014-05-24 23:52:44,2014-05,5,2424,86
1,40004,SGY1zM5vXy6T4rdB,2014-05-30 15:34:13,2014-05,5,2699,94
2,7377,rZ8By2I0wKNm1r0t,2014-05-21 13:23:07,2014-05,5,638,169
3,5792,SGY1zM5vXy6T4rdB,2014-05-30 11:27:00,2014-05,5,513,94
4,61756,cwXiTYqrj7AOGn7x,2014-05-28 23:41:59,2014-05,5,3772,127


In [13]:
# track ids from feature file
df = pd.read_csv(input_data_dir + features['emotion_embeddings'][1], sep='\t')
feat_track_ids = df['id'].values
feat_track_ids

array(['04q3VppIQEET5rzy', '07xF9Q0K1t3ist7K', '0JbYcELIqpMPvAHk', ...,
       'zzyyPUs7hC9Nz2e1', 'zzz0n04uuTUA7fNh', 'zzzj3LYaZtYtbzSr'],
      dtype=object)

In [14]:
# remove interactions with items not in the feature files
df_interactions = df_interactions[df_interactions['track_id'].isin(feat_track_ids)]
df_interactions.shape

(7916, 7)

In [15]:
#df_interactions[['user_id', 'track_id', 'rating', 'timestamp']].to_csv(os.path.join(output_dir, 'interactions.tsv'), index=False, sep='\t', header=False)
df_interactions[['user_id_int', 'track_id_int', 'rating', 'timestamp']].to_csv(os.path.join(output_dir, 'interactions.tsv'), index=False, sep='\t', header=False)

In [16]:
item_id_map = df_interactions[['track_id', 'track_id_int']].drop_duplicates()
item_id_map.set_index('track_id', inplace=True)
item_id_map.head()

,track_id_int
track_id,
Q08DjqYYbc3AvWQx,86
SGY1zM5vXy6T4rdB,94
rZ8By2I0wKNm1r0t,169
cwXiTYqrj7AOGn7x,127
td6odZZtNFj6Toou,180


In [17]:
del df_interactions

In [18]:
for feature_type in features:

    for file in features[feature_type]:
        if dataset == 'onion' and 'emma' in file:
            continue

        print('Processing', file)
        df = pd.read_csv(os.path.join(input_data_dir, file), index_col=0, sep='\t')
        df = df.merge(item_id_map, left_index=True, right_index=True)
        feature_name = file.split('.')[0].split('_')[1]
            
        folder_path = os.path.join(output_dir, feature_type, feature_name)

        if os.path.exists(folder_path):
            shutil.rmtree(folder_path)
        os.makedirs(folder_path, exist_ok=True)

        for index, row in tqdm.tqdm(df.iterrows()):
            np.save(os.path.join(folder_path, str(int(row['track_id_int'])) + '.npy'), row.values[:-1])

Processing id_emma.tsv


204it [00:00, 349.51it/s]


Processing id_gems.tsv


204it [00:00, 346.01it/s]


Processing id_maest.tsv


204it [00:00, 342.21it/s]


Processing id_bert.tsv


204it [00:00, 329.08it/s]


Processing id_resnet.tsv


204it [00:00, 227.73it/s]
